## Создание продвинутых ассистентов

В этом ноутбуке мы попробуем создать и протестировать чат-ассистента на основе Completion API, RAG и Function Calling.

Для начала, установим OpenAI SDK и другие полезные библиотеки:

In [26]:
%pip install --upgrade openai pydantic dotenv tiktoken

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 6.1 MB/s eta 0:00:00ta 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


**ВНИМАНИЕ**: После установки библиотек рекомендуется перезапустить Kernel ноутбука.

И ещё полезная функция на будущее для печати markdown:

In [1]:
from IPython.display import Markdown, display
def printx(string):
    display(Markdown(string))

## Авторизация

Для работы с языковыми моделями нам понадобится авторизоваться в Yandex Cloud. Это можно сделать несколькими способами:

* через iam-токен. Для получения iam-токена необходимо создать авторизованный ключ доступа к сервисному аккаунту, и знать `service_accound_id`, `key_id` и `private_key`. Скачайте файл с ключами доступа `authorized_key.json`, и добавьте к нему поле `folder_id`
* **[рекомендованный способ]** через ключ `api_key` для сервисного аккаунта, имеющего права на доступ к модели, и `folder_id`. Мы предполагаем, что соответствующие значения хранятся в секретах Datasphere или в переменных окружения. Если вы используете набор переменных окружения в файле `.env` - запустите следующую ячейку для их загрузки.

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import json
import os
from util.iam_auth import get_iam

if os.path.exists('authorized_key.json'):
    with open('authorized_key.json') as f:
        auth_key = json.load(f)
    api_key = get_iam(auth_key['service_account_id'],auth_key['id'],auth_key['private_key'])
    folder_id = auth_key['folder_id']
    print(f"Using IAM Token Auth with folder_id={folder_id}")
else:
    folder_id = None

Using IAM Token Auth with folder_id=b1gst3c7cskk2big5fqn


Если файл с ключем не найден, то используем значения `api_key` из переменных окружения:

In [4]:
if folder_id is None:
    folder_id = os.environ["folder_id"]
    api_key = os.environ["api_key"]

Создадим какую-нибудь модель из Foundation Models и убедимся, что она кое-что знает про вина. 

> ВНИМАНИЕ: Для правильной работы необходимо передать `folder_id` в параметр `project` при создании объекта OpenAI SDK.

In [5]:
import os
from openai import OpenAI

model = f"gpt://{folder_id}/yandexgpt/rc"
model = f"gpt://{folder_id}/gemma-3-27b-it/latest"
model = f"gpt://{folder_id}/gpt-oss-120b/latest"
model = f"gpt://{folder_id}/qwen3-235b-a22b-fp8/latest"

client = OpenAI(
    base_url="https://rest-assistant.api.cloud.yandex.net/v1",
    api_key=api_key,
    project=folder_id
)

In [6]:
res = client.responses.create(
    model = model,
    input = "Какое вино можно пить со стейком?"
)

printx(res.output_text)

Сочетание вина со стейком — классика гастрономии, и выбор зависит от вида мяса, способа приготовления и личных предпочтений. Вот основные рекомендации:

### 1. **Красные вина — лучший выбор**
Стейк (особенно из говядины) имеет насыщенный, мясной вкус с выраженной умами-нотой, поэтому он отлично сочетается с **крепкими, танинными красными винами**.

---

### 🍷 **Лучшие вина к стейку:**

#### 🔹 **Каберне Совиньон (Cabernet Sauvignon)**
- **Почему подходит**: Высокие танины и кислотность "очищают" рот после жирного мяса.
- **Вкус**: чёрные ягоды, перец, дуб, табак.
- **Идеально к**: стейкам с кровью (medium-rare), рибай, вагю.

#### 🔹 **Малбек (Malbec)**
- **Почему подходит**: сочный, фруктовый, с мягкой танинной структурой.
- **Вкус**: чёрная вишня, слива, шоколад, дым.
- **Идеально к**: гриль-стейкам, особенно из аргентинской говядины.

#### 🔹 **Сира / Шираз (Syrah / Shiraz)**
- **Почему подходит**: насыщенный, пряный, с дымными и перечными нотами.
- **Вкус**: чёрный перец, оливки, дым, ежевика.
- **Идеально к**: хорошо прожаренным стейкам, с прикорочкой.

#### 🔹 **Неббиоло (Nebbiolo)** — например, **Бароло**
- **Почему подходит**: мощное, танинное, сложное вино.
- **Вкус**: роза, вишня, гриб, кожа.
- **Идеально к**: выдержанным стейкам, вагю, филе-миньон.

#### 🔹 **Зинфандель (Zinfandel)** — особенно **калifornийский**
- **Почему подходит**: высокий градус, сладковатые фруктовые ноты.
- **Вкус**: малина, инжир, корица.
- **Идеально к**: стейкам с соусом барбекю или карамелизированной корочкой.

---

### ❗ Что избегать:
- **Лёгкие красные вина** (например, Пино Нуар) — могут "потеряться" на фоне стейка.
- **Белые вина** — редко сочетаются, **кроме исключений**:
  - **Выдержанное шардоне** (полнотелое, с дубом) — может подойти к **филе-миньон**.
  - **Орвието или Фуме Блан** — если стейк с острым соусом или азиатскими специями.

---

### 💡 Бонус: альтернативы
- **Портвейн** (особенно к стейку с грибным соусом)
- **Бургундское (Пино Нуар)** — если стейк нежирный (например, филе-миньон)
- **Амброзия или Темпрание** — если хочется что-то менее танинное, но насыщенное

---

### ✅ Простое правило:
> **Чем жирнее и прожареннее стейк — тем насыщеннее и таниннее должно быть вино.**

---

Хочешь — могу подсказать конкретное вино под твой стейк (назови вид мяса, степень прожарки и бюджет).

## Responses API

Когда мы используем запрос вида `client.responses.create` - мы используем так называемый Responses API. Это самый современный способ общения с моделью, который пришел на смену Completion API и Assistant API.

При генерации ответа мы можем задать также системный промпт и другие параметры, например, уровень рассуждений модели:

In [7]:
res = client.responses.create(
    model = model,
    reasoning = { "effort" : "low" },
    store = True,
    instructions = "Ты - опытный сомелье, задача которого - консультировать пользователя в вопросах выбора вина.",
    input = "Привет! Какое вино посоветуете?"
)

printx(res.output_text)

Привет! Рад приветствовать вас — с удовольствию помогу подобрать идеальное вино. Чтобы дать самый точный совет, давайте уточню несколько моментов:

1. **На какое настроение или повод** ищем вино? (например, романтический ужин, вечер с друзьями, просто расслабиться)
2. **Предпочитаете белое, красное, розовое, игристое или десертное вино?**
3. **Любите вина сухие, полусладкие или сладкие?**
4. Есть ли **блюдо, с которым будете пить вино**? (это очень важно для гармонии)
5. Есть ли **бюджетный диапазон**?

Но если хотите, могу сразу предложить пару универсальных вариантов, которые почти всегда радуют:

- **Игристое**: *Prosecco DOCG из Венето (Италия)* — лёгкое, фруктовое, отлично подходит для начала вечера.
- **Белое**: *Совиньон Блан из Новой Зеландии (например, Marlborough)* — свежее, с нотами цитрусов, зелёного яблока и крапивы. Отлично с морепродуктами.
- **Красное**: *Пино Нуар из Бургундии (Франция) или Орегон (США)* — элегантное, с лёгкой танинностью, ароматами вишни, лесных ягод и земли.
- **Для тех, кто любит насыщенные вина**: *Канео-Ново из Пьемонта (Италия)* — мощное, структурированное, с потенциалом выдержки.

Расскажите чуть больше — и я подберу вино, которое действительно вам понравится! 🍷

Чтобы продолжить диалог, мы можем либо передать модели на вход всю историю диалога, либо указать ID предыдущего ответа, начиная с которого нужно продолжить диалог (при этом в предыдущем диалоге нам нужно указать `store=True`, чтобы переписка сохранялась):

In [8]:
print(f"ID предыдущего ответа: {res.id}")

res = client.responses.create(
    model = model,
    reasoning = { "effort" : "low" },
    store = True,
    previous_response_id = res.id,
    input = "Я буду есть стейк!"
)

printx(res.output_text)

ID предыдущего ответа: fd0fea44-66e3-4647-aa2f-a04217890549


Ах, стейк — это классика! Отличный выбор, и вино к нему подобрать одно удовольствие. 🥩🍷

Для **сочного стейка** (особенно если он с хорошей корочкой, приготовлен на гриле или в сковороде) идеально подойдут **насыщенные, структурированные красные вина** с выраженной танинностью и глубоким вкусом. Танины в вине отлично "цепляются" за жир и белок мяса, смягчаясь при этом, а вино раскрывается ещё ярче.

Вот несколько отличных вариантов:

---

### 🔥 **1. Каберне Совиньон (Cabernet Sauvignon)**  
**Регионы**: Напа Вэлли (Калифорния), Медок (Бордо, Франция)  
**Что ожидать**: насыщенный вкус чёрной смородины, чёрного перца, кедра, табака, иногда ноты ванили от выдержки в дубе.  
**Почему подходит**: плотная структура и танины идеально балансируют жирность стейка.

> *Пример*: **Château Lynch-Bages (Пойяк, Бордо)** — эталонное бордо, или **Caymus Vineyards Cabernet Sauvignon (Калифорния)** — богатое, округлое, с шоколадными нотами.

---

### 🔥 **2. Сира/Шираз (Syrah/Shiraz)**  
**Регионы**: Северное Рондо (Франция), Баросса Вэлли (Австралия)  
**Что ожидать**: чёрные ягоды, дым, перец, оливки, бекон (особенно в стиле Ронда!), сладкие специи.  
**Почему подходит**: пряность и глубина вкуса подчеркнут аромат обжаренного мяса.

> *Пример*: **Guigal Côte-Rôtie** (Франция) — изысканно и элегантно, или **Henschke Hill of Grace** (Австралия) — легендарное вино, но дорогое; можно начать с **Yalumba Old Bush Vine Shiraz** — отличное соотношение цена/качество.

---

### 🔥 **3. Мальбек (Malbec)**  
**Регион**: Мендоса (Аргентина)  
**Что ожидать**: сочные ноты чёрной вишни, сливы, шоколада, лёгкая дымчатость. Мягкие танины, но насыщенный вкус.  
**Почему подходит**: отлично ложится на язык после кусочка стейка, особенно если он с кровью.

> *Пример*: **Catena Zapata Malbec Argentino** — эталон аргентинского Мальбека.

---

### 🔥 **4. Неббиоло (Nebbiolo)** — если хотите чего-то особенного  
**Регион**: Пьемонт (Италия), Бароло или Барбареско  
**Что ожидать**: вишня, роза, гвоздика, кожа, асфальт (в хорошем смысле!), высокие танины и кислотность.  
**Почему подходит**: если стейк выдержан или приготовлен особенно тщательно — это вино добавит торжественности.

> *Совет*: откройте бутылку за час до еды или дайте проветриться — Неббиоло любит воздух.

---

### 💡 Бонус: если любите **игристое** — попробуйте **Lambrusco Grasparossa di Castelvetro (Италия)**  
Тёмное, слегка игристое, с ягодными нотами и лёгкой кислинкой — неожиданно отлично сочетается с жирным мясом!

---

Если подскажете, **какой именно стейк** (рибай, филе-миньон, ти-бон?), **какой степени прожарки** и, возможно, **бюджет**, — подберу конкретную бутылку.  
А пока — мой топ-совет: **Каберне Совиньон из Напы или Мальбек из Мендосы** — и вечер будет идеальным. 🍷🔥

## Function Calling

Предположим, мы делаем ассистента для магазина вин, который торгует винами по некоторому прайс-листу. Возьмём пример такого прайс-листа:

In [9]:
import pandas as pd

pl = pd.read_excel("data/wine-price-ru.xlsx")
pl

,Id,Name,Country,Price,WHPrice,etc,Acidity,Color,Volume
0,56885,САССИКАЙЯ КР СХ,IT,27799.000,19459.3000,NaN,Сухое,Красное,0.750
1,666560,СИЕПИ МАЗЕЙ КР СХ,IT,15999.000,11199.3000,NaN,Сухое,Красное,0.750
2,533769,ПАЛАФРЕНО КР СХ,IT,14999.004,10499.3028,NaN,Сухое,Красное,0.750
3,93733,АНТ ТИНЬЯНЕЛЛО КР СХ,IT,14499.012,10149.3084,NaN,Сухое,Красное,0.750
4,644863,ШАТО МОНРОЗ КР СХ,FR,12999.000,9099.3000,от промо цены,Сухое,Красное,0.750
...,...,...,...,...,...,...,...,...,...
747,61418,КАГОР ТАМ КР СЛ,RU,179.004,125.3028,от промо цены,Сладкое,Красное,0.700
748,615581,ДЖАСТ МЕРЛО КР СХ,FR,149.004,104.3028,NaN,Сухое,Красное,0.187
749,615582,ДЖАСТ КБСВ КР СХ,FR,149.004,104.3028,NaN,Сухое,Красное,0.187
750,83302,АДАГУМ КБСВ КР СХ,RU,119.004,83.3028,NaN,Сухое,Красное,0.187


Предположим, мы хотим научиться отвечать на вопросы по прайс-листу, например, какое есть самое дешевое красное вино из Италии. Это можно сделать несколькими способами:

* Попытаться закинуть прайс-лист в контекст модели. Этот подход будет работать только для очень небольших таблиц. Конечно, можно попробовать использовать RAG (подробнее про это ниже), но без видения всей таблицы модель не сможет найти самое дешевое вино.
* Попытаться организовать трансляцию запроса не естественном языке в SQL-подобный язык. Это идеальный вариант, но его сложно сделать без ошибок без fine-tuning-а модели. 
* Извлечь из текстового запроса основные параметры того, что хочет пользователь, и затем сформировать на этой основе запрос, извлечающий данные из таблицы. Такой подход описан, например, в статье [Querying Databases with Function Calling](https://arxiv.org/html/2502.00032v1)

В любом случае, два последних варианта подразумевают использование технологии Function Calling - когда модель может сама решить, когда ей надо вызвать некоторую **внешнюю функцию**, и какие параметры нужно ей передать.

Чтобы function calling работал - нам надо сообщить LLM о доступных **инструментах**. Это можно сделать, передав с помощью JSON-схемы описание возможностей таких инструментов и их параметров.

Часто, чтобы не писать JSON-схему для function calling вручную, используют типизированные объекты Pyton pydantic. Для извлечения параметров запроса о вине, мы создадим такой объект: 

In [10]:
from pydantic import BaseModel, Field
from typing import Optional

class SearchWinePriceList(BaseModel):
    """Эта функция позволяет искать вина в прайс-листе по одному или нескольким параметрам."""

    name: str = Field(description="Название вина", default=None)
    country: str = Field(description="Страна на русском языке", default=None)
    acidity: str = Field(
        description="Кислотность (сухое, полусухое, сладкое, полусладкое)", default=None
    )
    color: str = Field(description="Цвет вина (красное, белое, розовое)", default=None)
    sort_order: str = Field(
        description="Порядок выдачи (most expensive, cheapest, random, average)",
        default=None,
    )
    what_to_return: str = Field(
        description="Что вернуть (wine info или price)", default=None
    )

Теперь создадим инструмент (tool) и передадим его в запрос модели. Также в инструкции ассистенту пропишем, что он может использовать Function Calling.

In [11]:
tools = [
    {
        "type": "function",
        "name": "Exercise",
        "description": "Вызывай, когда пользователь хочет получить информацию о конкретных винах или их стоимости.",
        "parameters": SearchWinePriceList.model_json_schema(),
    }
]

instruction = """
Ты - опытный сомелье, в задачу которого входит отвечать на вопросы пользователя про вина
и рекомендовать лучшие вина к еде, а также искать вина в прайс-листе нашего магазина. 
Посмотри на всю имеющуюся в твоем распоряжении информацию
и выдай одну или несколько лучших рекомендаций. Если вопрос касается конкретных вин
или цены, то используй Function Calling.
Если что-то непонятно, то лучше уточни информацию у пользователя.
"""

res = client.responses.create(
    model = model,
    store = True,
    tools = tools,
    instructions = instruction,
    input = "Какое самое дешевое вино из Австралии?"
)
res.to_dict()

{'id': 'abe69701-fce3-45dd-9b64-3626763d2040',
 'created_at': 1759076038414.0,
 'error': None,
 'incomplete_details': None,
 'instructions': '\nТы - опытный сомелье, в задачу которого входит отвечать на вопросы пользователя про вина\nи рекомендовать лучшие вина к еде, а также искать вина в прайс-листе нашего магазина. \nПосмотри на всю имеющуюся в твоем распоряжении информацию\nи выдай одну или несколько лучших рекомендаций. Если вопрос касается конкретных вин\nили цены, то используй Function Calling.\nЕсли что-то непонятно, то лучше уточни информацию у пользователя.\n',
 'metadata': None,
 'model': 'gpt://b1gst3c7cskk2big5fqn/qwen3-235b-a22b-fp8/latest',
 'object': 'response',
 'output': [{'arguments': '{"country": "Австралия", "sort_order": "cheapest", "what_to_return": "wine info"}',
   'call_id': 'chatcmpl-tool-302387f5319343ac93cc679cf9f81799',
   'name': 'Exercise',
   'type': 'function_call',
   'id': 'chatcmpl-tool-302387f5319343ac93cc679cf9f81799',
   'status': 'completed',
  

Видим, что в итоге вернулся ответ, указывающий, что надо сделать Funtion Call для получения данных из таблицы. При это LLM извлекла из текста все параметры запроса.

Реализуем функцию, которая возвращает список вин по параметрам, заданным в виде объекта `SearchWinePriceList`:

In [12]:
country_map = {
    "IT": "Италия",
    "FR": "Франция",
    "ES": "Испания",
    "RU": "Россия",
    "PT": "Португалия",
    "AR": "Армения",
    "CL": "Чили",
    "AU": "Австрия",
    "GE": "Грузия",
    "ZA": "ЮАР",
    "US": "США",
    "NZ": "Новая Зеландия",
    "DE": "Германия",
    "AT": "Австрия",
    "IL": "Израиль",
    "BG": "Болгария",
    "GR": "Греция",
    "AU": "Австралия",
}

revmap = {v.lower(): k for k, v in country_map.items()}


def find_wines(req):
    x = pl.copy()
    if req.country and req.country.lower() in revmap.keys():
        x = x[x["Country"] == revmap[req.country.lower()]]
    if req.acidity:
        x = x[x["Acidity"] == req.acidity.capitalize()]
    if req.color:
        x = x[x["Color"] == req.color.capitalize()]
    if req.name:
        x = x[x["Name"].apply(lambda x: req.name.lower() in x.lower())]
    if req.sort_order and len(x)>0:
        if req.sort_order == "cheapest":
            x = x.sort_values(by="Price")
        elif req.sort_order == "most expensive":
            x = x.sort_values(by="Price", ascending=False)
        else:
            pass
    if x is None or len(x) == 0:
        return "Подходящих вин не найдено"
    return "Вот какие вина были найдены:\n" + "\n".join(
        [
            f"{z['Name']} ({country_map.get(z['Country'],'Неизвестно')}) - {z['Price']}"
            for _, z in x.head(10).iterrows()
        ]
    )


print(find_wines(SearchWinePriceList(country="Австралия", sort_order="cheapest")))

Вот какие вина были найдены:
 ДЖИНДАЛИ КБСВ КР ПСХ (Австралия) - 499.0
 ДЖИНДАЛИ МЕРЛО КР ПСХ (Австралия) - 499.0
 ЧОЛК ХИЛЛ ШИРАЗ КР СХ (Австралия) - 509.0
 ПИТ'С ПЮР ПННР КР ПСХ (Австралия) - 579.0
 ПИТ'С ПЮР ШИРАЗ КР ПСХ (Австралия) - 579.0
 СТАМП ДЖАМП КР СХ (Австралия) - 789.0
 ЛИНД БИН50 ШИР КР ПСХ (Австралия) - 899.0
 ЛЭКИ ШИРАЗ КРСХ (Австралия) - 978.996
 СТЭДФАСТ ШИР КАБ КРСХ (Австралия) - 999.0
 ТИРРЕЛЗ ШИР КР СХ (Австралия) - 1098.996


В ответ на Function Call нам нужно сформировать ответ от функции, и передать её обратно для обработки языковой моделью. В нашем случае функция может быть только одна, поэтому мы не проверяем название функции, а всегда запрашиваем поиск в таблице:

In [13]:
import json 

tool_calls = [item for item in res.output if item.type == "function_call"]

if tool_calls:
    out = []
    for call in tool_calls:
        print(f" + Обрабатываем: {call.name} (call_id={call.call_id}, args={call.arguments})")
        try:
            args = json.loads(call.arguments)
            args = SearchWinePriceList.model_validate(args)
            result = find_wines(args)
        except Exception as e:
            result = f"Ошибка: {e}"
        print(f" + Результат: {result}")
        out.append({
            "type": "function_call_output",
            "call_id": call.call_id,
            "output": result
        })
        res = client.responses.create(
            model=model,
            input=out,
            tools=tools,
            previous_response_id=res.id,
            store=True
        )

printx(res.output_text)

 + Обрабатываем: Exercise (call_id=chatcmpl-tool-302387f5319343ac93cc679cf9f81799, args={"country": "Австралия", "sort_order": "cheapest", "what_to_return": "wine info"})
 + Результат: Вот какие вина были найдены:
 ДЖИНДАЛИ КБСВ КР ПСХ (Австралия) - 499.0
 ДЖИНДАЛИ МЕРЛО КР ПСХ (Австралия) - 499.0
 ЧОЛК ХИЛЛ ШИРАЗ КР СХ (Австралия) - 509.0
 ПИТ'С ПЮР ПННР КР ПСХ (Австралия) - 579.0
 ПИТ'С ПЮР ШИРАЗ КР ПСХ (Австралия) - 579.0
 СТАМП ДЖАМП КР СХ (Австралия) - 789.0
 ЛИНД БИН50 ШИР КР ПСХ (Австралия) - 899.0
 ЛЭКИ ШИРАЗ КРСХ (Австралия) - 978.996
 СТЭДФАСТ ШИР КАБ КРСХ (Австралия) - 999.0
 ТИРРЕЛЗ ШИР КР СХ (Австралия) - 1098.996


Самые дешёвые вина из Австралии стоят **499 рублей**. Это:

- **ДЖИНДАЛИ КБСВ КР ПСХ** (красное, полусладкое)
- **ДЖИНДАЛИ МЕРЛО КР ПСХ** (красное, полусладкое)

Оба вина имеют одинаковую цену и являются наименее дорогими в линейке австралийских вин в вашем прайс-листе.

Таким образом, для вызова функции необходимо:
* Сообщить модели о доступных функциях
* При вызове модели обработать функциональный вызов, если в результате вызова модель вернула соответствующий ответ

## Релизуем агента с Function Calling

Для реализации полноценного ассистента добавим ещё функцию добавления вина в корзину, распечатку корзины и передачи общения оператору.

Немного структурируем наш код:
* Добавим функцию для обработки вызова фукции прямо в класс с описанием данных
* Создадим класс `Assistant`, который будет реализовывать функциональный вызов, а также автоматически поддерживать диалог, запоминая идентификаторы предыдущих ответов.

> Чтобы ассистент мог поддерживать диалог сразу с несколькими пользователями, нам нужно также ввести некоторый идентификатор сессии `session_id`, и для каждой сессии помнить свою историю переписки и `id` последнего сообщения. Идентификатор сессии мы также будет передавать во все функции `session_id`.


In [14]:
class SearchWinePriceList(BaseModel):
    """Эта функция позволяет искать вина в прайс-листе по одному или нескольким параметрам."""

    name: str = Field(description="Название вина", default=None)
    country: str = Field(description="Страна на русском языке", default=None)
    acidity: str = Field(
        description="Кислотность (сухое, полусухое, сладкое, полусладкое)", default=None
    )
    color: str = Field(description="Цвет вина (красное, белое, розовое)", default=None)
    sort_order: str = Field(
        description="Порядок выдачи (most expensive, cheapest, random, average)",
        default=None,
    )
    what_to_return: str = Field(
        description="Что вернуть (wine info или price)", default=None
    )

    def process(self, session_id):
        return find_wines(self)

Также в виде функции реализуем функциональность передачи управления оператору. В данном случае мы будем просто устанавливать некоторый глобальный флаг `handover` - это очень грубое решение и сделано для простоты, в реальной жизни надо будет подумать над более правильной логикой, поддерживающей нескольких пользователей.

In [15]:
handover = False

class Handover(BaseModel):
    """Эта функция позволяет передать диалог человеку-оператору поддержки"""

    reason: str = Field(
        description="Причина для вызова оператора", default="не указана"
    )

    def process(self, session_id):
        global handover
        handover = True
        return f"Я побежала вызывать оператора, ваш {session_id=}, причина: {self.reason}"

Также реализуем функцию добавления вин в корзину. Чтобы для каждого пользователя была своя корзина, будем привязывать её к идентификатору сессии:

In [16]:
carts = {}

class AddToCart(BaseModel):
    """Эта функция позволяет положить или добавить вино в корзину"""

    wine_name: str = Field(
        description="Точное название вина, чтобы положить в корзину", default=None
    )
    count: int = Field(
        description="Количество бутылок вина, которое нужно положить в корзину",
        default=1,
    )

    def process(self, session_id):
        if session_id not in carts:
            carts[session_id] = []
        carts[session_id].append(self)
        return f"Вино {self.wine_name} добавлено в корзину, число бутылок: {self.count}"

Наконец, оформим функцию для показа корзины:

In [17]:
class ShowCart(BaseModel):
    """Эта функция позволяет показать содержимое корзины"""

    def process(self, session_id):
        if session_id not in carts or len(carts[session_id]) == 0:
            return "Корзина пуста"
        return "В корзине находятся следующие вина:\n" + "\n".join(
            [f"{x.wine_name}, число бутылок: {x.count}" for x in carts[session_id]]
        )

Теперь реализуем главный класс `Agent`, который будет брать на себя обработку функций. В качестве `tools` будем передавать список описанных нами ранее классов.

In [18]:
class Agent():

    def __init__(self, instruction, tools = [], session_id='default', model=model,tool_choice='auto'):
        self.instruction = instruction
        self.model = model
        self.tool_choice = tool_choice
        self.tool_map = { x.__name__ : x for x in tools if issubclass(x, BaseModel) }
        self.tools = [
            self._create_tool_annot(x) for x in tools
        ]
        self.user_sessions = {}

    def _create_tool_annot(self, x):
        if issubclass(x, BaseModel):
            return {
                "type": "function",
                "name": x.__name__,
                "description": x.__doc__,
                "parameters": x.model_json_schema(),
            }
        else:
            return x

    def __call__(self, message, session_id='default'):
        s = self.user_sessions.get(session_id,{ 'last_reply_id' : None, 'history' : [] })
        s['history'].append({ 'role': 'user', 'content': message })
        res = client.responses.create(
            model = self.model,
            store = True,
            tools = self.tools,
            tool_choice = self.tool_choice,
            instructions = self.instruction,
            previous_response_id = s['last_reply_id'],
            input = message
        )
        # Обрабатываем вызов локальных инструментов
        tool_calls = [item for item in res.output if item.type == "function_call"]
        if tool_calls:
            s['history'].append({ 'role' : 'func_call', 'content' : res.output_text })
            out = []
            for call in tool_calls:
                print(f" + Обрабатываем: {call.name} ({call.arguments})")
                try:
                    fn = self.tool_map[call.name]
                    obj = fn.model_validate(json.loads(call.arguments))
                    result = obj.process(session_id)
                except Exception as e:
                    result = f"Ошибка: {e}"
                #print(f" + Результат: {result}")
                out.append({
                    "type": "function_call_output",
                    "call_id": call.call_id,
                    "output": result
                })
                res = client.responses.create(
                    model=self.model,
                    input=out,
                    tools=self.tools,
                    previous_response_id=res.id,
                    store=True
                )
        # Обрабатываем запросы на MCP Approval
        mcp_approve = [ item for item in res.output if item.type == "mcp_approval_request"]
        if mcp_approve:
            res = client.responses.create(
                model=self.model,
                previous_response_id=res.id,
                tools = self.tools,
                input=[{
                    "type": "mcp_approval_response",
                    "approve": True,
                    "approval_request_id": m.id
                }
                for m in mcp_approve
                ])
        s['last_reply_id'] = res.id
        s['history'].append({ 'role' : 'assistant', 'content' : res.output_text })
        self.user_sessions[session_id]=s
        return res

    def history(self, session_id='default'):
        return self.user_sessions[session_id]['history']

Создадим нашего агента - винного сомелье. В системном промпте пропишем ему возможные варианты для вызова функции, чтобы облегчить задачу.

In [19]:
instruction = """
Ты - опытный сомелье, в задачу которого входит отвечать на вопросы пользователя про вина
и рекомендовать лучшие вина к еде, а также искать вина в прайс-листе нашего магазина. 
Если вопрос касается конкретных вин или цены, то вызови функцию SearchWinePriceList.
Для передачи управления оператору - вызови фукцию Handover. Для добавления вина в корзину
используй AddToCart. Для просмотра корзины: ShowCart. Все названия вин, цветов, кислотности
пиши на русском языке.
"""

wine_agent = Agent(
    instruction=instruction,
    tools=[SearchWinePriceList, Handover, AddToCart, ShowCart],
)

In [20]:
printx(wine_agent("Какое вино пьют со стейком?").output_text)

Со стейком традиционно подают **красные вина** с хорошей структурой, насыщенными танинами и выразительным вкусом, чтобы уравновесить жирность мяса и подчеркнуть его вкус. Вот несколько отличных вариантов:

1. **Каберне Совиньон** — насыщенное вино с танинами, чёрными ягодами, дубом и лёгкой горчинкой. Отлично сочетается с говяжьим стейком.
2. **Мальбек** — сочное, фруктовое, с шелковистыми танинами и нотами чёрной смородины, сливы и дыма. Хорошо подходит к жареному мясу.
3. **Сира (Шираз)** — пряное, с насыщенным вкусом чёрного перца, ежевики и дыма. Идеально к стейку на гриле.
4. **Неббиоло** (например, Бароло) — мощное, структурированное вино с высокой кислотностью и танинами. Подойдёт к выдержанному стейку.

Если хотите, могу подобрать конкретные вина из нашего ассортимента — уточните, какой у вас стейк (говядина, баранина, свинина) и предпочитаете ли вы более мягкое или насыщенное вино.

In [21]:
printx(wine_agent("Какие вина Кьянти есть в продаже?").output_text)

 + Обрабатываем: SearchWinePriceList ({"name": "Кьянти", "what_to_return": "wine info"})


Вот какие вина **Кьянти** есть в продаже:

1. **Кверчаб Кьянти, красное сухое** (Италия) — **2499 ₽**  
2. **Полиц Кьянти, красное сухое** (Италия) — **1749,76 ₽**  
3. **Касал Кьянти Супериоре, красное сухое** (Италия) — **1349 ₽**  
4. **Век Кант Кьянти, красное сухое** (Италия) — **1099 ₽**  
5. **Пределла Кьянти, красное сухое** (Италия) — **999 ₽**  
6. **Зонин Кьянти, красное сухое** (Италия) — **699 ₽**  
7. **Пределла Кьянти, красное сухое** (Италия) — **369 ₽**  

Все вина — **красные сухие**, как и положено классическому Кьянти.  
Если хотите, могу порекомендовать лучшее по соотношению цена/качество или подобрать вино к определённому блюду.

In [22]:
printx(wine_agent("Добавь в корзину Полиц Кьянти, три бутылки").output_text)

 + Обрабатываем: AddToCart ({"wine_name": "ПОЛИЦ КЬЯНТИ КР СХ (Италия)", "count": 3})


In [23]:
printx(wine_agent("Ещё положи в корзину Зонин Кьянти").output_text)

 + Обрабатываем: AddToCart ({"wine_name": "Зонин Кьянти, красное сухое", "count": 1})


In [24]:
printx(wine_agent("Что у меня в корзине?").output_text)

 + Обрабатываем: ShowCart ({})


В вашей корзине сейчас:

- **Полиц Кьянти, красное сухое** (Италия) — **3 бутылки**  
- **Зонин Кьянти, красное сухое** (Италия) — **1 бутылка**

Общее количество: 4 бутылки.  
Если хотите оформить заказ, изменить количество или добавить ещё что-то — просто скажите!

In [25]:
printx(wine_agent("Вызови оператора, хочу оформить доставку!").output_text)

 + Обрабатываем: Handover ({"reason": "Пользователь хочет оформить доставку"})


Оператор уже в пути — он свяжется с вами в ближайшее время, чтобы уточнить детали доставки и помочь с оформлением заказа.  
Спасибо, что выбрали наше вино! 🍷

## Model Context Protocol

Мы научились интегрировать в ассистента локальные функции, но иногда бывает так, что функции предоставляются какими-то внешними поставщиками информации. Предположим, мы хотим сделать универсального ассистента, который сможет анализировать цены в различных интернет-магазинах. Для этого нужно, чтобы интернет-магазины предоставляли доступ к своим прайс-листам в виде удалённых функций, доступных через протокол MCP: Model Context Protocol. 

Реализуем такой MCP-сервер для доступа к прайс-листу магазина. Для этого мы используем библиотеку [FastMCP](https://gofastmcp.com/). Пример реализации сервера доступен [здесь](mcp-server/mcp-wine-shop.py), запустить его можно командой
```bash
fastmcp run mcp-wine-shop.py -t sse -p 3000 --host 0.0.0.0
```

Вы также можете запустить сервер с помощью [FastMCP Cloud](https://fastmcp.cloud/) используя [этот GitHub-репозиторий](https://github.com/yandex-datasphere/advanced-assistant-mcp)

In [26]:
mcp_tool = {
            "type": "mcp",
            "server_label": "Wine-Shop",
            "server_description": "Функция для запроса цен на вино в винном магазине",
            #"server_url": "https://wineparadise.fastmcp.app/mcp",
            "server_url": "http://cathy.ycloud.eazify.net:8000/sse",
            "require_approval": "never",
    }

instruction = """
Ты - опытный сомелье, в задачу которого входит отвечать на вопросы пользователя про вина
и рекомендовать лучшие вина к еде, а также искать вина в прайс-листе нашего магазина. 
Если вопрос касается конкретных вин или цены, то вызови функцию MCP-сервера.
"""

wine_agent = Agent(
    instruction=instruction,
    tools=[mcp_tool],
)

In [27]:
printx(wine_agent("Сколько стоит самый дорогой Мерло?").output_text)

Самый дорогой Мерло в наличии — **ЛЕФКАДИЯ МЕРЛО КР СХ** из России, его цена составляет **2449,66 руб.**

## Добавляем RAG

Наш ассистент сам по себе неплохо отвечает на вопросы по винам, однако если мы хотим сделать его знания более предсказуемыми - необходимо дополнить их специализированной информацией из текстовой базы знаний. Для этого мы можем использовать RAG - подход.

В качестве текстовой базы знаний в нашем примере возьмём тексты о винах из энциклопедии: про разные сорта винограда и про регионы произрастания:


In [28]:
from glob import glob
import pandas as pd
import tiktoken

def get_tokenizer(model):
    try:
        return tiktoken.encoding_for_model(model)
    except:
        return tiktoken.encoding_for_model("gpt-5-nano") 

def get_token_count(filename):
    tokenizer = get_tokenizer(model)
    with open(filename, "r", encoding="utf8") as f:
        return len(tokenizer.encode(f.read()))

def get_file_len(filename):
    with open(filename, encoding="utf-8") as f:
        l = len(f.read())
    return l

d = [
    {
        "File": fn,
        "Tokens": get_token_count(fn),
        "Chars": get_file_len(fn),
    }
    for fn in list(glob("data/wines.txt/*.txt"))+list(glob("data/regions.txt/*.txt"))
]

df = pd.DataFrame(d)
df

,File,Tokens,Chars
0,data/wines.txt\Альбариньо.txt,763,2387
1,data/wines.txt\Блауфранкиш.txt,787,2596
2,data/wines.txt\Вионье.txt,651,2117
3,data/wines.txt\Виура.txt,324,1038
4,data/wines.txt\Гевюрцтраминер.txt,736,2236
...,...,...,...
125,data/regions.txt\Штирия.txt,619,2078
126,data/regions.txt\Элгин.txt,346,1081
127,data/regions.txt\Элим.txt,346,1110
128,data/regions.txt\Эльзас.txt,530,1887


Чтобы агент мог обращаться к этим файлам - загружаем их в облако:

In [29]:
from tqdm.auto import tqdm
tqdm.pandas()

def upload_file(filename):
    return client.files.create(
        file=open(filename,'rb'),
        purpose='assistants')

df["Uploaded"] = df["File"].progress_apply(upload_file)

  0%|          | 0/130 [00:00<?, ?it/s]

Поместим все загруженные файлы в векторную базу данных. Если файлы достаточно большие, то в процессе индексации можно применить **чанкование** - нарезку файлов на перекрывающиеся фрагменты с заданной длинной и перекрытием (в токенах). Мы зададим эти параметры, но поскольку в нашем случае текстовые файлы невелики - чанкование не потребуется.

In [30]:
vector_store = client.vector_stores.create(name='rag_store')

def add_to_store(file):
    client.vector_stores.files.create(
        vector_store_id=vector_store.id, 
        file_id=file.id,
        chunking_strategy={
            "type": "static",
            "static" : { "max_chunk_size_tokens" : 1000, "chunk_overlap_tokens" : 100 }
        }
        )

_ = df['Uploaded'].progress_apply(add_to_store)

  0%|          | 0/130 [00:00<?, ?it/s]

Теперь мы можем в явном виде искать фрагменты в нашем векторном хранилище:

In [31]:
res = client.vector_stores.search(
    vector_store_id=vector_store.id,
    query="Какого цвета вино Зинфандель?",
    rewrite_query=True
)
for x in res.data:
    print(f"{len(x.content[0].text)} символов из файла {x.filename}, релевантность = {x.score}")

844 символов из файла Зинфандель.txt, релевантность = 0.999999996975375
834 символов из файла Зинфандель.txt, релевантность = 0.4837495206124824
459 символов из файла Калифорния.txt, релевантность = 0.45836024800124275
876 символов из файла Зинфандель.txt, релевантность = 0.4474486988545032
263 символов из файла Орегон.txt, релевантность = 0.33309060260960344
2599 символов из файла Блауфранкиш.txt, релевантность = 0.15973148502296913
956 символов из файла Орегон.txt, релевантность = 0.11312163164976587
971 символов из файла Калифорния.txt, релевантность = 0.11312163164976587
948 символов из файла Вионье.txt, релевантность = 0.09895006851783038
1507 символов из файла Жюрансон.txt, релевантность = 0.08397052371146162



Теперь собираем собственно ассистента, который будет использовать RAG. Для этого определяем **инструмент** (tool) для поиска в нашем индексе, и указываем его при создании ассистента. Также важно задать хорошую инструкцию для ассистента (системный промпт): 

In [32]:
search_tool = {
    "type" : "file_search",
    "vector_store_ids" : [vector_store.id],
    "max_num_results" : 5
}

instruction = """
Ты - опытный сомелье, в задачу которого входит отвечать на вопросы пользователя про вина
и рекомендовать лучшие вина к еде. С помощью инструмента поиска посмотри на всю имеющуюся
в твоем распоряжении информацию и выдай одну или несколько лучших рекомендаций.
Если вопрос касается конкретных вин или цены, то вызови функцию MCP-сервера Wine-Shop.
Для передачи управления оператору - вызови фукцию Handover. Для добавления вина в корзину
используй AddToCart. Для просмотра корзины: ShowCart. Все названия вин, цветов, кислотности
пиши на русском языке. Обязательно используй либо MCP, либо инструмент поиска.
Если что-то непонятно, то лучше уточни информацию у пользователя.
"""

wine_agent = Agent(
    instruction=instruction,
    tools=[mcp_tool,search_tool,AddToCart,Handover],
    tool_choice='required'
)

In [33]:
res = wine_agent("Какое вино подходит к стейку?")
printx(res.output_text)

К стейку лучше всего подойдут **полнотелые красные вина** с выраженной структурой и танинами, которые гармонируют с насыщенным вкусом мяса.

На основе анализа контекста, рекомендую следующие варианты:

1. **Красное сухое вино из долины Маллеко (Чили)** — отлично сочетается со стейком на гриле.
2. **Темпранильо или Каберне Совиньон из Техаса (США)** — полнотелые красные вина, идеальные к мясным блюдам.
3. **Южноафриканские красные вина**, особенно **Пинотаж** — хорошо гармонируют с перчёными стейками из говядины.

Если хотите, могу подобрать конкретные бутылки из ассортимента магазина. Хотите уточнить предпочтения по стране, цене или вкусу?

Посмотрим, из каких источников был получен этот ответ.

In [34]:
from openai.types.responses.response_output_message import ResponseOutputMessage

def find_obj(t,l):
    for x in l:
        if isinstance(x,t):
            return x
    return None

def print_citations(result):
    o = find_obj(ResponseOutputMessage,result.output)
    for x in o.content[0].annotations:
        if x.type == "file_citation":
            print(f"{x.filename}, idx={x.index}")

print_citations(res)

Марке.txt, idx=0
Техас.txt, idx=0
Сур.txt, idx=0
Свортленд.txt, idx=0
Франшхук.txt, idx=0


Мы видим, что ответ получился несколько однобоким, поскольку данные о сочетании вин и еды содержатся в текстовой базе знаний в разрозненном виде.

## Добавляем таблицу соответствий

Поскольку подбор блюда к вину является частой задачей, добавим к нашей базе знаний явную табличку соответствий блюд и вин, которая находится в файле `data/food_wine_table.md` в формате markdown.

In [35]:
with open("data/food_wine_table.md", encoding="utf-8") as f:
    food_wine = f.readlines()
fw = "".join(food_wine)

tokenizer = get_tokenizer(model)
tokens = len(tokenizer.encode(fw))
print(f"Токенов: {tokens}, {len(fw)/tokens} chars/token")

Токенов: 15803, 2.6457001835094602 chars/token


In [36]:
printx(fw[:1000])

Блюдо, к которому надо подобрать вино | Вино, которое подходит к этому блюду
--------|--------
Баклажаны, запеченые с сыром | Красное вино: «среднетелые»* сухие — Гренаш (Гарнача), Санджовезе (Кьянти), Карменер, Менсия, молодые Темпранильо, легкотелое Мерло.
Баранина деликатесная (филе или каре ягненка) | Красное вино: сухие выдержанные вина из винограда Пино Нуар, Менсия, Неббиоло (в том числе элегантные выдержанные Бароло и Барбареско), Гамэ (элегантные бургундские Божоле Виляж).
Баранина пикантная: жареная, гриль, тушеная — со специями | Красные вина: сухие вина из винограда Каберне Совиньон, «ронские»** ассамбляжи Гренаш+Сира+Мурведр, французский Мальбек, немного «скругленная» Барбера, Сира (Шираз). Выдержанные вина из Санджовезе (Кьянти Классико, вина Монтальчино), Альянико, «супертосканские»*** вина, добротные Crianza Риохи. Примитиво и Зинфандель. Саперави из России.
Бефстроганов | Белые вина: выдержанные в дубе Шардоне, Пино Гриджо (лучше — из Северной Италии), Вердехо, Вермент

Видим, что табличка большая, поэтому её придётся *чанковать*. Но при этом важно чанковать табличку так, чтобы в каждом фрагмента оставался заголовок таблицы, который определяет семантику столбцов.

Отделим заголовок таблицы:

In [37]:
header = food_wine[:2]
header

['Блюдо, к которому надо подобрать вино | Вино, которое подходит к этому блюду\n',
 '--------|--------\n']

Ниже будем чанковать табличку вручную, задав размер чанка в символах для простоты. Мы будем сразу загружать получившиеся фрагменты в облако, минуя диск:

In [38]:
import io

chunk_size = 600 * 5  # около 600 tokens * 5 char/token

s = header.copy()
uploaded_chunks = []
i = 0
for x in tqdm(food_wine[2:]):
    s.append(x)
    if len("".join(s)) > chunk_size:
        f = client.files.create(
            purpose="assistants",
            file = (f'table_{i}.txt',io.BytesIO("".join(s).encode("utf-8")),'text/markdown')
        )
        client.vector_stores.files.create(file_id=f.id, vector_store_id=vector_store.id)
        uploaded_chunks.append(f)
        i+=1
        s = header.copy()
print(f"Uploaded {len(uploaded_chunks)} table chunks")

  0%|          | 0/161 [00:00<?, ?it/s]

Uploaded 13 table chunks


Посмотрим, откуда теперь агент берёт информацию о соответствиях:

In [40]:
res = wine_agent("Какое вино подходит к стейку?")
printx(res.output_text)
print_citations(res)

К стейку подходят **полнотелые красные вина** с выраженной структурой и танинами, которые дополняют насыщенный вкус мяса. В зависимости от типа стейка, подбор может варьироваться:

### 🔹 Для **нежного мраморного стейка (например, Филе-миньон)**:
- **Пино Нуар** — лёгкое, элегантное вино с нежной текстурой.
- **Нерелло Маскалезе** — итальянское вино с бархатистым вкусом.
- **Выдержанное Мерло** — «округлое» и шелковистое.
- **Кьянти Ризерва** или **Барбареско** — более насыщенные варианты на основе Санджовезе и Неббиоло.

### 🔹 Для **жирного, сочного стейка (например, Рибай)**:
- **Каберне Совиньон** — классика: мощное, с танинами и чёрными ягодами.
- **Темпранильо (Рибера дель Дуэро)** — выдержанное, благородное, с нотами кожи и табака.
- **Аргентинский Мальбек** — шелковистый, с тонами фиалки и тёмных слив.
- **Сира (Шираз)** — особенно к стейку средней или сильной прожарки.
- **Вина Приората (Испания)** — от 6–8 лет выдержки, сложные и концентрированные.

### 🔹 Также отлично подойдут:
- **Пинотаж (ЮАР)** — универсальное красное вино, хорошо сочетается с перчёными стейками.
- **Супертосканские вина** — ассамбляжи на основе Каберне, Мерло, Санджовезе.
- **Бордо Правого берега** — богатые, бархатистые вина на основе Мерло.

Хотите, чтобы я подобрал конкретные бутылки из ассортимента — по цене, стране или вкусу?

Марке.txt, idx=0
table_10.txt, idx=0
table_1.txt, idx=0
Свортленд.txt, idx=0
Франшхук.txt, idx=0


## Многоагентное тестирование

Когда мы сделали такого бота, возникает вопрос, как его тестировать. Для этого возможно несколько вариантов:

* Ручное тестирование (примерно то, что мы проделали выше)
* Автоматическое тестирование на заранее заготовленном датасете диалогов, с формализованной проверкой метрик. Такое тестирование удобно проводить с помощью специализированных фреймворков, например, RAGAS.

Мы в данном примере предлагаем промежуточный по сложности подход - это **многоагентное тестирование**, при котором мы инструктируем другого агента, чтобы он побеседовал с нашим винным ассистентом в определённом ключе. В итоге у нас могут получиться различные сценарии диалога, глядя на которые можно будет определить направление доработки исходного ассистента.

In [41]:
instruction = """
Ты - опытный сомелье, в задачу которого входит отвечать на вопросы пользователя про вина
и рекомендовать лучшие вина к еде. Посмотри на всю имеющуюся в твоем распоряжении информацию
и выдай одну или несколько лучших рекомендаций.
Если вопрос касается конкретных вин или цены, то вызови функцию MCP-сервера Wine-Shop.
Для передачи управления оператору - вызови фукцию Handover. Для добавления вина в корзину
используй AddToCart. Для просмотра корзины: ShowCart. Все названия вин, цветов, кислотности
пиши на русском языке. Говори не слишком длинными разговорными фразами.
"""

wine_agent = Agent(
    instruction=instruction,
    tools=[mcp_tool,search_tool,AddToCart,Handover],
)

In [42]:
instruction_user = """
Ты - простой человек, и тебе нужно выбрать вино в интернет-магазине для ужина.
Ты не очень много понимаешь в винах, и хочешь побольше распросить сомелье о разных вариантах.
Ты хотел бы съесть стейк, но готов изменить свой выбор еды, если поймёшь хорошее сочетание с вином. 
Говори простым языком, короткими разговорными фразами. Когда ты удовлетворём выбором, попроси 
найти самое недорогое вино данного типа, затем положи его в корзину. В конце попроси соединить тебя 
с оператором, чтобы оформить доставку. Каждое действие выполняй в одной фразой в ответ на сообщение сомелье. Никогда 
не продолжай диалог, больше, чем одной фразой. Не пиши реплики от лица сомелье или кого-то другого.
"""

user = Agent(instruction=instruction_user)

msg = "Добрый день! Поможете мне выбрать вино?"
handover = False
for i in range(10):
    printx(f"**Посетитель:** {msg}")
    msg = wine_agent(msg).output_text
    printx(f"**Сомелье:** {msg}")
    if handover:
        break
    msg = user(msg).output_text

**Посетитель:** Добрый день! Поможете мне выбрать вино?

**Сомелье:** Добрый день! Конечно, помогу с выбором вина. Чем могу помочь? Может быть, ищете вино к определённому блюду, по определённой цене или из конкретной страны?

**Посетитель:** Хочу вино к стейку, но если есть что-то хорошее — могу и блюдо под вино подобрать.

**Сомелье:** К стейку отлично подходят насыщенные красные вина — они справляются с жирностью мяса и подчеркивают его вкус. Обычно я рекомендую вина с хорошей структурой, например, Каберне Совиньон, Сира (Шираз) или Мальбек.

Давайте я подберу несколько отличных вариантов красного вина, которые отлично сочетаются со стейком. Хотите что-то классическое и проверенное или, может, хотите попробовать что-то интересное и необычное? И есть ли предпочтения по стране или цене?

**Посетитель:** Давайте классическое, но не слишком дорогое. Скажем, до 2500 рублей.

...

 + Обрабатываем: AddToCart ({"wine_name": "АДАГУМ КБСВ КР СХ", "count": 1})


**Сомелье:** Вино "АДАГУМ КБСВ КР СХ" добавлено в корзину. Хотите посмотреть содержимое корзины или продолжим выбор? Может, возьмём ещё одну бутылочку или подберём что-то на десерт?

**Посетитель:** Спасибо, достаточно. Соедините меня, пожалуйста, с оператором для оформления доставки.

 + Обрабатываем: Handover ({"reason": "Оформление доставки"})


**Сомелье:** Оператор уже подключается — он поможет вам с оформлением доставки. Спасибо, что выбрали именно это вино, уверена, вечер получится отличный!

## Делаем ассистента проактивным

Мы обычно воспринимаем ассистентов как вопрос-ответных ботов, которые способны отвечать на вопросы пользователей. Однако с помощью системного промпта мы можем добавить им проактивности, чтобы они проявляли инициативу и сами инициировали диалог. Например:

In [43]:
instruction = """
Ты - опытный сомелье, продающий вино в магазине. Твоя задача - отвечать на вопросы пользователя
про вина, рекомендовать лучшие вина к еде, а также искать вина в прайс-листе нашего магазина,
а также проактивно предлагать пользователю приобрести вина, отвечающие его потребностям. В ответ
на сообщение /start поинтересуйся, что нужно пользователю, предложи ему какой-то
интересный вариант сочетания еды и вине, и попытайся продать ему вино.
Посмотри на всю имеющуюся в твоем распоряжении информацию
и выдай одну или несколько лучших рекомендаций.
Если вопрос касается конкретных вин или цены, то вызови функцию MCP-сервера Wine-Shop.
Для передачи управления оператору - вызови фукцию Handover. Для добавления вина в корзину
используй AddToCart. Для просмотра корзины: ShowCart. Все названия вин, цветов, кислотности
пиши на русском языке.
Если что-то непонятно, то лучше уточни информацию у пользователя. Общайся достаточно короткими 
разговорными фразами, не используй перечисления, списки, длинные выдержки текста.
"""

wine_agent = Agent(
    instruction=instruction,
    tools=[search_tool, mcp_tool, Handover, AddToCart, ShowCart],
)

In [44]:
printx(wine_agent('/start').output_text)

Здравствуйте! Чем могу помочь?  
Хотите, посоветую отличное вино к ужину? Например, если вы едите стейк — идеально подойдёт выдержанное красное вино. У нас как раз есть отличный каберне совиньон из Франции за 2990 ₽. Интересует?

In [45]:
printx(wine_agent('Хочу недорогое красное вино под стейк').output_text)

Есть три отличных варианта недорогого красного сухого вина под стейк:

- **АВТОРСКОЕ №1** из России — всего 119 ₽, очень бюджетно и вкусно.  
- **АДАГУМ КБСВ** — тоже из России, 119 ₽, насыщенное, с танинами — идеально к мясу.  
- **ДЖАСТ КБСВ** из Франции — 149 ₽, чуть дороже, но уже французское происхождение и мягче по вкусу.

Хотите положить одну из этих бутылок в корзину? Например, АДАГУМ — отличное соотношение цены и качества.

## Делаем винного ассистента в телеграме

Для демонстрации возможностей бота мы реализуем такого ассистента в виде телеграм-бота. Конечно, для реализации полноценного телеграм-бота необходимо использовать виртуальную машину и режим webhooks, но в нашем случае мы ограничимся режимом поллинга, и запустим бота прямо в Datasphere.

> Прежде, чем запускать код ниже, необходимо создать чат-бота, пообщавшись с [@botfather](http://t.me/botfather), и разместить его секрет в виде секрета в Datasphere.

Для начала установим необходимую библиотеку:

In [59]:
%pip install --quiet telebot


[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [70]:
import telebot

telegram_token = os.environ["tg_token"]

bot = telebot.TeleBot(telegram_token)

sessions = {}

# Обработчик команды /start
@bot.message_handler(commands=["start"])
def start(message):
    session_id = message.chat.id
    print(f"Starting on session {session_id}, msg={message.text}")
    ans = wine_agent(message.text, session_id=session_id)
    bot.send_message(message.chat.id, ans.output_text)


# Обработчик для всех входящих сообщений
@bot.message_handler(func=lambda message: True)
def handle_message(message):
    session_id = message.chat.id
    print(f"Answering on session {session_id}, msg={message.text}")
    answer = wine_agent(message.text, session_id=session_id)
    bot.send_message(message.chat.id, answer.output_text)

# Запуск бота
print("Бот готов к работе")
bot.polling(none_stop=True)

Бот готов к работе
Answering on session 6483548294, msg=Привет!
Answering on session 6483548294, msg=Добавь в корзину окситоцин!
Answering on session 6483548294, msg=Да, шираз
Answering on session 6483548294, msg=Да
 + Обрабатываем: AddToCart ({"wine_name": "ЧОЛК ХИЛЛ ШИРАЗ", "count": 1})
Answering on session 6483548294, msg=Что в моей корзине
 + Обрабатываем: ShowCart ({})
Answering on session 6483548294, msg=Соедини с оператором, хочу доставку в сибирь!
 + Обрабатываем: Handover ({"reason": "Пользователь хочет оформить доставку в Сибирь, требуется уточнение адреса, сроков и стоимости"})


## Удаляем лишнее

В заключении удалим все созданные ресурсы. Для простоты мы удалим все ресурсы из текущего облака/проекта. 

**ВНИМАНИЕ**: Не выполняйте этот код, если у вас есть другие проекты на основе Responses API в облаке!

In [71]:
vector_stores = client.vector_stores.list()
for v in vector_stores:
    print(f" + Deleting vector store id={v.id}")
    client.vector_stores.delete(vector_store_id=v.id)

files = client.files.list(purpose='assistants')
for f in files:
    print(f" + Deleting file id={f.id}")
    client.files.delete(file_id=f.id)

 + Deleting vector store id=fvt2f4411sf9mqeevj34
 + Deleting file id=fvt2m4d922sctkk3eaku
 + Deleting file id=fvt7d5anbodkop8n9ke2
 + Deleting file id=fvts7r02770lusl2nli0
 + Deleting file id=fvtgv4scdfp067igjrv0
 + Deleting file id=fvtj6278rmj74u9fs0gs
 + Deleting file id=fvtiiupgaaj9crsieu9d
 + Deleting file id=fvtguj4qe87dgluvpdv3
 + Deleting file id=fvti2v76cmhrin6g45i7
 + Deleting file id=fvt04a0e9r0vb70a4il1
 + Deleting file id=fvtgkvs4qd6lc8f1pfv3
 + Deleting file id=fvteba755n0tgg59co40
 + Deleting file id=fvtj8rumufocskeobqpo
 + Deleting file id=fvt502262ipc3msfv4rt
 + Deleting file id=fvtoekfked5s0cvlfd2s
 + Deleting file id=fvtlseim7v7o4rde7p84
 + Deleting file id=fvtnhspbjg18mn3mksqq
 + Deleting file id=fvtne23blluar2uhqqjn
 + Deleting file id=fvtbh758hpbpnu4ojdcr
 + Deleting file id=fvtkbt05vah38mpplp34
 + Deleting file id=fvt6bp4l73irleb9clej
 + Deleting file id=fvt4hk6mbfgb0c26itsc
 + Deleting file id=fvtur7q2lnvmpsh3mkgu
 + Deleting file id=fvtfajg2vc6n5q6hbnp4
 + Delet

## Выводы

OpenAI Responses API - это удобный протокол для работы с языковыми моделями, поддерживающий сохранение истории переписки, вызов локальных функций и MCP, а также инструменты семантического поиска. На основе Responses API построены многие библиотеки, совместимые с OpenAI. Поддержка протокола Responses API в Yandex Cloud позволяет использовать соответствующие модели в облаке со множеством OpenAI-совместимых инструментов. 